In [1]:
import pysam
from Bio import SeqIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from tqdm import tqdm

In [34]:
def get_error_matrix(bam_path, bam_c_path, ref_path, contig='gi|49175990|ref|NC_000913.2|'):
    matrix = np.zeros((2, 3), dtype=np.int64)
    bam = pysam.AlignmentFile(bam_path, 'rb')
    bam_c = pysam.AlignmentFile(bam_c_path, 'rb')
    pup = bam.pileup(contig)
    pup_c = bam_c.pileup(contig)
    ref_seq = next(SeqIO.parse(ref_path, 'fasta')).seq
    for c, c_corr in tqdm(zip(pup, pup_c)):
        ref_base = ref_seq[c.reference_pos]
        c_dict = dict(zip(c.get_query_names(), c.get_query_sequences()))
        c_corr_dict = dict(zip(c_corr.get_query_names(), c_corr.get_query_sequences()))
        for name, base in c_dict.items():
            if base.upper() == ref_base: # Correct base in raw data
                row = 1
            else: # Error in raw data
                row = 0
            if name in c_corr_dict:
                if c_corr_dict[name].upper() == 'N': # Base is absent in corrected reads
                    column = 2
                elif c_corr_dict[name].upper() == ref_base: # Correct base in corrected reads
                    column = 1
                else: # Error in corrected reads
                    column = 0
            else: # Base is absent in corrected reads
                column = 2
            matrix[row][column] += 1
    return pd.DataFrame(matrix, index=['Error in raw data', 'Correct base in raw data'], columns=['Error in corrected reads', 'Correct base in corrected reads', 'Base is absent in corrected reads'])

## e.coli 10k

In [7]:
bam_path = '/media/data/NGS_4/e_10_sorted.bam'
ref_path = '/media/data/NGS_4/MG1655-K12.first10K.fasta'

_TRIMMOMATIC_

`TrimmomaticPE ecoli_10K_err_1.fastq.gz ecoli_10K_err_2.fastq.gz e_10_1_trim.fastq e_10_1_un.fastq e_10_2_trim.fastq e_10_2_un.fastq LEADING:30 TRAILING:30 SLIDINGWINDOW:10:30  MINLEN:30`

Input Read Pairs: 29639 Both Surviving: 24954 (84,19%) Forward Only Surviving: 2200 (7,42%) Reverse Only Surviving: 1649 (5,56%) Dropped: 836 (2,82%)


In [8]:
bam_c_path = '/media/data/NGS_4/e_10_trim_sorted.bam'

In [36]:
get_error_matrix(bam_path, bam_c_path, ref_path)

10000it [00:11, 858.38it/s]


,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,2214,2,6227
Correct base in raw data,2,4042830,1184813


_Spades_

`./spades.py -1 /media/data/NGS_4/ecoli_10K_err_1.fastq.gz -2 /media/data/NGS_4/ecoli_10K_err_2.fastq.gz --only-error-correction -o /media/data/NGS_4/e_10_corr/
`

In [37]:
bam_c_path = '/media/data/NGS_4/e_10_sorted_spades.bam'

In [38]:
get_error_matrix(bam_path, bam_c_path, ref_path)

10000it [00:12, 829.51it/s]


,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,1562,6175,706
Correct base in raw data,11,5168796,58838


## e.coli 400k

In [41]:
bam_path = '/media/data/NGS_4/e_400_sorted.bam'
ref_path = '/media/data/NGS_4/MG1655-K12.first400K.fasta'

_TRIMMOMATIC_

Input Read Pairs: 1381602 Both Surviving: 1182531 (85,59%) Forward Only Surviving: 92495 (6,69%) Reverse Only Surviving: 68702 (4,97%) Dropped: 37874 (2,74%)

In [44]:
bam_c_path = '/media/data/NGS_4/e_400_sorted_trim.bam'

In [45]:
get_error_matrix(bam_path, bam_c_path, ref_path)

400000it [08:13, 810.37it/s] 


,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,160824,172,288286
Correct base in raw data,139,193570075,54986656


_Spades_

`./spades.py -1 /media/data/NGS_4/ecoli_400K_err_1.fastq.gz -2 /media/data/NGS_4/ecoli_400K_err_2.fastq.gz --only-error-correction -o /media/data/NGS_4/e_400_corr/
`

In [46]:
bam_c_path = '/media/data/NGS_4/e_400_sorted_spades.bam'

In [47]:
get_error_matrix(bam_path, bam_c_path, ref_path)

400000it [08:49, 756.11it/s] 


,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,146015,267551,35716
Correct base in raw data,331,244508576,4047963
